# Clustering Consulting Project 

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

In [1]:
import findspark
findspark.init('/home/mint/spark-2.1.0-bin-hadoop2.7')
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('cluster').getOrCreate()
data = spark.read.csv('hack_data.csv', inferSchema=True, header=True)
data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



In [15]:
from pyspark.ml.feature import VectorAssembler, StandardScaler
final_data = VectorAssembler(
    inputCols=['Session_Connection_Time', 'Bytes Transferred', 'Kali_Trace_Used',
               'Servers_Corrupted', 'Pages_Corrupted', 'WPM_Typing_Speed'],
    outputCol='features').transform(data)
final_data2 = StandardScaler(inputCol='features', outputCol='scaledFeatures').fit(final_data).transform(final_data)
# final_data.show()

In [16]:
from pyspark.ml.clustering import KMeans
model_2 = KMeans(featuresCol='scaledFeatures', k=2).setSeed(1).fit(final_data2)
print('wssse')
print(model_2.computeCost(final_data2))

wssse
601.7707512676716


In [17]:
model_3 = KMeans(featuresCol='scaledFeatures', k=3).setSeed(1).fit(final_data2)
print('wssse')
print(model_3.computeCost(final_data2))

wssse
434.75507308487647


In [23]:
model_2.transform(final_data2).groupBy('prediction').count().orderBy('prediction').show()
model_3.transform(final_data2).groupBy('prediction').count().orderBy('prediction').show()

+----------+-----+
|prediction|count|
+----------+-----+
|         0|  167|
|         1|  167|
+----------+-----+

+----------+-----+
|prediction|count|
+----------+-----+
|         0|  167|
|         1|   88|
|         2|   79|
+----------+-----+

